In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.callbacks import ModelCheckpoint


In [2]:

train_dir = 'data/train'
val_dir = 'data/val'

train_datagen = ImageDataGenerator(rescale=1./255, 
                                   shear_range=0.2,                               
                                   zoom_range=0.2, 
                                   horizontal_flip=True)

val_datagen = ImageDataGenerator(rescale=1./255)


train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(224, 224), 
                                                    batch_size=32, 
                                                    class_mode='binary')

val_generator = val_datagen.flow_from_directory(val_dir,
                                                target_size=(224, 224), 
                                                batch_size=32, 
                                                class_mode='binary')


In [30]:
vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers
for layer in vgg_base.layers:
    layer.trainable = False


NameError: name 'VGG16' is not defined

In [31]:

model = Sequential()

model.add(vgg_base)


model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))  
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               12845568  
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 513       
Total params: 27,560,769
Trainable params: 12,846,081
Non-trainable params: 14,714,688
_________________________________________________________________


In [32]:
model.compile(optimizer=Adam(lr=0.0001),
              loss='binary_crossentropy', 
              metrics=['accuracy'])


c:\Users\ADMIN\anaconda3\envs\ml_env\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [33]:
checkpoint = ModelCheckpoint('accident_detection_model.h5',  
                             monitor='val_accuracy',  
                             save_best_only=True, 
                             mode='max', 
                             verbose=1) 

In [34]:

history = model.fit(train_generator,
                    steps_per_epoch=train_generator.samples // 32,
                    validation_data=val_generator,
                    validation_steps=val_generator.samples // 32,
                    epochs=30,
                    callbacks=[checkpoint])  


Epoch 1/30
24/24 [==============================] - 248s 10s/step - loss: 0.8211 - accuracy: 0.5731 - val_loss: 0.5956 - val_accuracy: 0.6458

Epoch 00001: val_accuracy improved from -inf to 0.64583, saving model to accident_detection_model.h5
Epoch 2/30
24/24 [==============================] - 275s 12s/step - loss: 0.5980 - accuracy: 0.6812 - val_loss: 0.5236 - val_accuracy: 0.7396

Epoch 00002: val_accuracy improved from 0.64583 to 0.73958, saving model to accident_detection_model.h5
Epoch 3/30
24/24 [==============================] - 276s 12s/step - loss: 0.5441 - accuracy: 0.6996 - val_loss: 0.4749 - val_accuracy: 0.7812

Epoch 00003: val_accuracy improved from 0.73958 to 0.78125, saving model to accident_detection_model.h5
Epoch 4/30
24/24 [==============================] - 273s 11s/step - loss: 0.5292 - accuracy: 0.7352 - val_loss: 0.4840 - val_accuracy: 0.7500

Epoch 00004: val_accuracy did not improve from 0.78125
Epoch 5/30
24/24 [==============================] - 273s 11s/ste

In [3]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='train_accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.legend()
plt.show()

NameError: name 'history' is not defined

In [ ]:

model = load_model('accident_detection_model.h5')


In [ ]:

def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))  
    img_array = image.img_to_array(img)  
    img_array = np.expand_dims(img_array, axis=0)  
    img_array /= 255.0 
    return img_array


In [ ]:

def predict_image(model, img_path):

    processed_image = preprocess_image(img_path)

    prediction = model.predict(processed_image)

    if prediction < 0.5:
        return "Accident", prediction
    else:
        return "Non-Accident", prediction


In [ ]:
img_path = 'testtimage1.jpg' 

label, probability = predict_image(model, img_path)

print(f"Predicted Label: {label}, Probability: {probability}")


1/1 [==============================] - 1s 581ms/step
Predicted Label: Non-Accident, Probability: [[0.936163]]
